In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import TimestampType

CATALOGO_DESTINO = "v_credit"
SCHEMA_DESTINO = "silver"
TABELA_DESTINO = "tb_pesquisa"

nome_tabela_destino = f"{CATALOGO_DESTINO}.{SCHEMA_DESTINO}.{TABELA_DESTINO}"

In [0]:
query = f"""
CREATE OR REPLACE TABLE {full_table_name_destination} (
    cd_pesquisa BIGINT COMMENT 'Identificador da pesquisa.',
    cd_chamado BIGINT COMMENT 'Vínculo com o chamado avaliado.',
    nu_nota SMALLINT COMMENT 'Nota atribuída (ex: 1 a 5).',
    dt_ingestion TIMESTAMP,
    dc_origem STRING,
    CONSTRAINT pk_pesquisa PRIMARY KEY (cd_pesquisa)
)
COMMENT 'Tabela silver de histórico de pesquisa de satisfação';
"""

spark.sql(query)
print(f"✅ Tabela {nome_tabela_destino} criada com sucesso!")